## API-Based Weather Data Processing and Export to Excel

#### This notebook uses an API to get weather forecast data for a city, processes it, and saves it into an Excel file 

#### Simple Explanation of the Code:

| Step | Description |
|------|-------------|
| **1. Import Libraries** | - `requests`: Fetches weather data from the internet.<br> - `pandas`: Organizes and processes the data into tables.<br> - `json`: Formats and handles the data from the API. |
| **2. Connect to the Weather API** | - Use an API key to access weather forecast data from OpenWeatherMap.<br> - Build the URL with the city ID and other parameters. |
| **3. Fetch Weather Data** | - Send a request to the API to get the weather data.<br> - Convert the response (JSON format) into Python-friendly data (lists, dict). |
| **4. Print the Data** | - Display the raw weather data in a neat and readable format using `json.dumps(data, indent=4)` for better readability. |
| **5. Extract Specific Information** | - Gather useful data like temperature, humidity, weather description, wind speed, etc., for each time slot.<br> - Store this extracted data in a list. |
| **6. Create a Table (DataFrame)** | - Use pandas to organize the extracted data into a structured table with rows and columns. |
| **7. Filter by Time (12 PM)** | - Select rows where the weather data is recorded at 12:00 PM every day. |
| **8. Separate Weather Types** | - Split the data into separate tables for:<br>   - Snowy conditions.<br>   - Rainy conditions.<br>   - Cloudy conditions. |
| **9. Save to Excel** | - Save all the data into an Excel file with multiple sheets:<br>   - A sheet for the full weather data.<br>   - Separate sheets for Snow, Rain, and Clouds. |
| **10. Final Output** | - Print a confirmation that the data has been saved into the Excel file. |




In [134]:
# Import necessary libraries
import requests  # To send HTTP requests
import pandas as pd  # To handle data and create DataFrames
import json  # To handle JSON data

# Install required libraries (in case you need them)
!pip install pandas
!pip install requests

In [135]:
# Define the API key to authenticate the request
API_key = 'd8bfc2a558b8aab911f2e4674842e261'

# Create the URL for making the request to the OpenWeather API
# The URL contains the API key, a city ID (524901 corresponds to Moscow), and the weather forecast endpoint
url = f'http://api.openweathermap.org/data/2.5/forecast?id=524901&appid={API_key}'

# Print the constructed URL to verify it
print(url)

# Send an HTTP GET request to the OpenWeather API using the constructed URL
response = requests.get(url)

# Print the response object to check the status and content of the response
print(response)

http://api.openweathermap.org/data/2.5/forecast?id=524901&appid=d8bfc2a558b8aab911f2e4674842e261
<Response [200]>


In [136]:
# Converts the JSON response into a Python dictionary
data = response.json()

In [137]:
# This will show the structure of the response and can be used to check the contents of the response.
#print(data)

In [138]:
print(type(data)) ## This helps to confirm that the response was parsed correctly and is now in dictionary format

# Use json.dumps() to pretty-print the JSON data with an indentation of 4 spaces.
# This will format the JSON data in a more human-readable way, which can be useful for viewing the entire structure.
print(json.dumps(data, indent=4))


<class 'dict'>
{
    "cod": "200",
    "message": 0,
    "cnt": 40,
    "list": [
        {
            "dt": 1737039600,
            "main": {
                "temp": 273.8,
                "feels_like": 268.55,
                "temp_min": 273.8,
                "temp_max": 273.97,
                "pressure": 1015,
                "sea_level": 1015,
                "grnd_level": 994,
                "humidity": 80,
                "temp_kf": -0.17
            },
            "weather": [
                {
                    "id": 500,
                    "main": "Rain",
                    "description": "light rain",
                    "icon": "10n"
                }
            ],
            "clouds": {
                "all": 100
            },
            "wind": {
                "speed": 5.94,
                "deg": 266,
                "gust": 13.77
            },
            "visibility": 10000,
            "pop": 1,
            "rain": {
                "3h": 0.54
          

In [139]:
# Create an empty list to store weather data
weather_data = []

# Loop through the data and extract useful weather information
for item in data['list']:
    weather_data.append({
        "datetime": item.get("dt_txt", ""),  # Date and time of the forecasted data
        "temp": item.get("main", {}).get("temp", 0),  # Temperature
        "feels_like": item.get("main", {}).get("feels_like", 0),  # Feels like temperature
        "temp_min": item.get("main", {}).get("temp_min", 0),  # Minimum temperature
        "temp_max": item.get("main", {}).get("temp_max", 0),  # Maximum temperature
        "pressure": item.get("main", {}).get("pressure", 0),  # Atmospheric pressure
        "humidity": item.get("main", {}).get("humidity", 0),  # Humidity percentage
        "weather_main": item.get("weather", [{}])[0].get("main", ""),  # Main weather condition (e.g., Rain, Snow)
        "weather_description": item.get("weather", [{}])[0].get("description", ""),  # Detailed weather description
        "clouds": item.get("clouds", {}).get("all", 0),  # Cloudiness percentage
        "wind_speed": item.get("wind", {}).get("speed", 0),  # Wind speed
        "wind_deg": item.get("wind", {}).get("deg", 0),  # Wind direction (degrees)
        "visibility": item.get("visibility", 0),  # Visibility in meters
        "pop": item.get("pop", 0),  # Probability of precipitation
        "snow_3h": item.get("snow", {}).get("3h", 0),  # Snowfall in the last 3 hours (if any)
    })

In [140]:
# Create a pandas DataFrame
df = pd.DataFrame(weather)

# Display the DataFrame
print(df.head())



              datetime    temp  feels_like  temp_min  temp_max  pressure  \
0  2025-01-16 12:00:00  273.42      268.19    273.41    273.42      1016   
1  2025-01-16 15:00:00  273.60      268.30    273.60    273.97      1015   
2  2025-01-16 18:00:00  274.17      268.97    274.17    274.54      1014   
3  2025-01-16 21:00:00  274.84      270.15    274.84    274.84      1013   
4  2025-01-17 00:00:00  275.14      270.10    275.14    275.14      1011   

   humidity weather_main weather_description  clouds  wind_speed  wind_deg  \
0        70         Snow          light snow     100        5.71       255   
1        78         Rain          light rain     100        5.94       266   
2        86         Rain          light rain     100        6.05       277   
3        97       Clouds     overcast clouds     100        5.38       283   
4        96       Clouds     overcast clouds     100        6.27       277   

   visibility  pop  snow_3h  
0       10000  0.2     0.13  
1       10000 

In [141]:
#dfilter the weather data at 12:00

# Ensure the 'datetime' column is in datetime format
df['datetime'] = pd.to_datetime(df['datetime'])

# Filter rows where the time is '12:00:00'
filtered_df = df[df['datetime'].dt.strftime('%H:%M:%S') == '12:00:00']

# Print the title and filtered data
print("Weather Data for 12:00:00 Time")
print(filtered_df)

# Write the filtered data to a CSV file
filtered_df.to_csv('filtered_weather_data_12_00.csv', index=False)

# Inform the user that the data has been saved
print("Filtered data has been written to 'filtered_weather_data_12_00.csv'!")


Weather Data for 12:00:00 Time
               datetime    temp  feels_like  temp_min  temp_max  pressure  \
0   2025-01-16 12:00:00  273.42      268.19    273.41    273.42      1016   
8   2025-01-17 12:00:00  275.47      270.84    275.47    275.47      1015   
16  2025-01-18 12:00:00  275.03      269.49    275.03    275.03      1009   
24  2025-01-19 12:00:00  272.98      268.15    272.98    272.98      1016   
32  2025-01-20 12:00:00  274.21      271.05    274.21    274.21      1023   
40  2025-01-16 12:00:00  273.42      268.19    273.41    273.42      1016   
48  2025-01-17 12:00:00  275.47      270.84    275.47    275.47      1015   
56  2025-01-18 12:00:00  275.03      269.49    275.03    275.03      1009   
64  2025-01-19 12:00:00  272.98      268.15    272.98    272.98      1016   
72  2025-01-20 12:00:00  274.21      271.05    274.21    274.21      1023   
87  2025-01-17 12:00:00  275.47      270.84    275.47    275.47      1015   
95  2025-01-18 12:00:00  275.03      269.49  

In [142]:
# Filter the data based on the weather conditions
snow_df = df[df['weather_main'] == 'Snow']
rain_df = df[df['weather_main'] == 'Rain']
clouds_df = df[df['weather_main'] == 'Clouds']

# Specify the file path
file_path = 'weather_data.xlsx'

# Install xlsxwriter
!pip install xlsxwriter

# Write the full weather data to the 'Full Data' sheet and filtered data to separate sheets
with pd.ExcelWriter(file_path, engine='xlsxwriter') as writer:
    # Save the full weather data to the 'Full Data' sheet
    df.to_excel(writer, sheet_name='Full Data', index=False)

    # Save the filtered Snow data to the 'Snow' sheet
    snow_df.to_excel(writer, sheet_name='Snow', index=False, header=True)

    # Save the filtered Rain data to the 'Rain' sheet
    rain_df.to_excel(writer, sheet_name='Rain', index=False, header=True)

    # Save the filtered Clouds data to the 'Clouds' sheet
    clouds_df.to_excel(writer, sheet_name='Clouds', index=False, header=True)

print("Data has been written to the Excel file with multiple sheets!")

Data has been written to the Excel file with multiple sheets!
